In [88]:
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import KFold

In [89]:
assess_cols = ['Analytical thinking', 'Systemic thinking', 'Adaptability', 'Focus']

In [90]:
users_df = pd.read_csv('../data/raw/user.csv', index_col='user_id')
results_df = pd.read_csv('../data/raw/results.csv', index_col='id')
submit_df = pd.read_csv('../data/to_sibmit.csv', index_col='id')

# teammate_assessments_df = pd.read_csv('../data/preprocessed/teammate_assessments.csv', index_col='user_id')
# team_results_df = pd.read_csv('../data/preprocessed/user_with_team_results.csv', index_col='user_id')

decisions_df = pd.read_csv('../data/preprocessed/user_decision_one_hot_all.csv', index_col='user_id')

In [91]:
# looks like doesn't improve performance
# users_df = users_df.merge(teammate_assessments_df, left_index=True, right_index=True)
# users_df = users_df.merge(team_results_df, left_index=True, right_index=True)

users_df = users_df.merge(decisions_df, left_index=True, right_index=True)

In [92]:
users_df.head()

,team_id,game_id,386,25,27,26,276,329,331,330,...,347,9,8,216,231,7,233,346,261,258
user_id,,,,,,,,,,,,,,,,,,,,,
10189,1664,235,True,True,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
10018,1690,237,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10580,1394,194,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10600,1908,262,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10112,1592,219,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False


In [93]:
users_known_df = results_df[[]].merge(users_df, left_index=True, right_index=True)
users_test_df = submit_df[[]].merge(users_df, left_index=True, right_index=True)

In [94]:
kf = KFold(n_splits=4, shuffle=True, random_state=42)
clf = RandomForestClassifier(
    # n_estimators=1000,
    # criterion='entropy',
    # max_depth=10,
    # max_features=0.9,
    # class_weight='balanced',
    )
recall = 0
to_submit_df = submit_df[[]].copy()

for column in assess_cols:
    for train_index, valid_index in kf.split(users_known_df):
        X_train, X_valid = users_known_df.iloc[train_index,:], users_known_df.iloc[valid_index,:]
        y_train, y_valid = results_df.iloc[train_index,:][column], results_df.iloc[valid_index,:][column]
        model = clone(clf).fit(X_train, y_train)
        pred = model.predict(X_valid)
        recall += recall_score(y_valid, pred, labels=[1.0,2.0,3.0,4.0,5.0,6.0], average='micro')
    model_full = clone(clf).fit(users_known_df, results_df[column])
    to_submit_df[column] = model_full.predict(users_test_df)
    # print(f'Model for column {column}: {model.__dict__}')

print(recall/32.)
to_submit_df.to_csv('../data/submission/random_forest.csv')

0.3106617647058823
